REMEMBER: FIRST CREATE A COPY OF THIS FILE WITH A UNIQUE NAME AND DO YOUR WORK THERE. AND MAKE SURE YOU COMMIT YOUR CHANGES TO THE `hw3_submissions` BRANCH.

# Assignment 3 | Cleaning and Exploring Data with Pandas



<img src="data/scoreCard.jpg" width=250>

In this assignment, you will investigate restaurant food safety scores for restaurants in San Francisco. Above is a sample score card for a restaurant. The scores and violation information have been made available by the San Francisco Department of Public Health. 

## Loading Food Safety Data


There are 2 files in the data directory:
1. business.csv containing food establishments in San Francisco
1. inspections.csv containing retaurant inspections records

Let's start by loading them into Pandas dataframes.  One of the files, business.csv, has encoding (ISO-8859-1), so you will need to account for that when reading it.

### Question 1

#### Question 1a
Read the two files noted above into two pandas dataframes named `bus` and `ins`, respectively. Print the first 5 rows of each to inspect them.


In [1]:
import pandas as pd

In [2]:
bus = pd.read_csv('data/businesses.csv', encoding='ISO-8859-1')
ins = pd.read_csv('data/inspections.csv')

In [3]:
bus.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494
3,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN
4,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051


In [4]:
ins.head()

,business_id,score,date,type
0,10,82,20160503,routine
1,10,94,20140729,routine
2,10,92,20140114,routine
3,19,94,20160513,routine
4,19,94,20141110,routine


## Examining the Business data

From its name alone, we expect the `businesses.csv` file to contain information about the restaurants. Let's investigate this dataset.

### Question 2

#### Question 2a: How many records are there?

In [40]:
num_rows= bus.shape[0]
print("Number of rows in Business dataframe = ", num_rows)

Number of rows in Business dataframe =  6315


#### Question 2b: How many unique business IDs are there?  

In [35]:
n = bus.business_id.nunique()
print("number of unique business IDs =",n)

number of unique business IDs = 6315


#### Question 2c: What are the 5 most common businesses by name, and how many are there in San Francisco?

In [39]:
n = 5
bus['name'].value_counts()[:n]

name
STARBUCKS COFFEE              72
PEET'S COFFEE & TEA           24
MCDONALDS                     12
SAN FRANCISCO SOUP COMPANY    11
WALGREENS                     11
Name: count, dtype: int64

## Zip code

Next, let's  explore some of the variables in the business table. We begin by examining the postal code.

### Question 3

#### Question 3a
How are the zip code values stored in python (i.e. data type)?

To answer this you might want to examine a particular entry.

In [49]:
dtypes = bus.dtypes['postal_code']
dtypes
print (bus['postal_code'].dtype)

object


In [57]:
type("postal_code")

str

#### Question 3b

What are the unique values of postal_code?

In [58]:
bus['postal_code'].value_counts()

postal_code
94110        585
94103        558
94102        458
94107        451
94133        417
94109        383
94111        282
94122        270
94118        243
94115        238
94114        230
94105        226
94108        224
94117        197
94112        190
94123        169
94124        164
94121        161
94132        134
94104        133
94116         93
94134         74
94127         71
94131         48
94158         17
94130          8
94143          5
CA             3
Ca             3
94101          2
94013          2
94129          2
94014          2
941033148      1
94602          1
94621          1
94188          1
00000          1
94609          1
95105          1
941            1
94120          1
941102019      1
94545          1
94066          1
92672          1
Name: count, dtype: int64

#### Question 3c

Let's say we decide to exclude the businesses that have no zipcode for our analysis (which might include food trucks for example).  Use the list of valid 5-digit zip codes below to create a new dataframe called bus_valid, with only businesses whose postal_codes show up in this list of valid zipcodes. How many businesses are there in this new dataframe?

In [61]:
validZip = ["94102", "94103", "94104", "94105", "94107", "94108",
            "94109", "94110", "94111", "94112", "94114", "94115",
            "94116", "94117", "94118", "94121", "94122", "94123", 
            "94124", "94127", "94131", "94132", "94133", "94134"]

In [62]:
vb = bus[bus['postal_code'].isin(validZip)]

In [67]:
num_rows2= vb.shape[0]
print("Number of rows in Valid Zipcode Business dataframe = ", num_rows2)

Number of rows in Valid Zipcode Business dataframe =  5999


## Latitude and Longitude

Another aspect of the data we want to consider is the prevalence of missing values. If many records have missing values then we might be concerned about whether the nonmissing values are representative of the population.

### Question 4
 
Consider the longitude and latitude in the business DataFrame. 

#### Question 4a

How many businesses are missing longitude values, working with only the businesses that are in the list of valid zipcodes?

In [72]:
null_latlong = vb[pd.isnull(vb['longitude'])]
null_latlong.shape[0]

2483

#### Question 4b

Create a new dataframe with one row for each valid zipcode. The dataframe should include the following three columns:

1. `postal_code`: Contains the zip codes in the `validZip` variable above.
2. `null_lon`: The number of businesses in that zipcode with missing `longitude` values.
3. `not_null_lon`: The number of businesses without missing `longitude` values.

In [83]:
new_df = pd.DataFrame(columns=['postal_code', 'null_lon', 'not_null_lon'])

In [84]:
for code in validZip:
    temp_df = vb[vb['postal_code'] == code]
    null_lon = temp_df['longitude'].isnull().sum()
    not_null_lon = temp_df['longitude'].notnull().sum()
    data = pd.DataFrame({'postal_code': [code], 'null_lon': [null_lon], 'not_null_lon': [not_null_lon]})
    new_df = new_df.append(data, ignore_index=True)
    
    new_df

AttributeError: 'DataFrame' object has no attribute 'append'

#### 4c. Do any zip codes appear to have more than their 'fair share' of missing longitude? 

To answer this, you will want to compute the proportion of missing longitude values for each zip code, and print the proportion missing longitude, and print the top five zipcodes in descending order of proportion missing postal_code.


In [85]:
new_df['proportion_missing'] = new_df['null_lon'] / (new_df['null_lon'] + new_df['not_null_lon'])

new_df = new_df.sort_values('proportion_missing', ascending=False)

print(new_df.head(5))

Empty DataFrame
Columns: [postal_code, null_lon, not_null_lon, proportion_missing]
Index: []


In [86]:
ins.head()

,business_id,score,date,type
0,10,82,20160503,routine
1,10,94,20140729,routine
2,10,92,20140114,routine
3,19,94,20160513,routine
4,19,94,20141110,routine


# Investigate the inspection data

Let's now turn to the inspection DataFrame. Earlier, we found that `ins` has 4 columns, these are named `business_id`, `score`, `date` and `type`.  In this section, we determine the granularity of `ins` and investigate the kinds of information provided for the inspections. 

### Question 5

#### Question 5a
As with the business data, assess whether there is one inspection record for each business, by counting how many rows are in the data and how many unique businesses there are in the data. If they are exactly the same number, it means there is only one inspection per business, clearly.

In [88]:
num_rows3= ins.shape[0]
print("Number of rows in Inspection dataframe = ", num_rows)
m = ins.business_id.nunique()
print("number of unique business IDs in inspection =",m)

Number of rows in Inspection dataframe =  6315
number of unique business IDs in inspection = 5730


#### Question 5b

What values does `type` take on? How many occurrences of each value is in the DataFrame? Create a new dataframe named `ins2` by copying `ins` and keeping only records with values of `type` that occur more than 10 times in the original table. In other words, eliminate records that have values of `type` that occur rarely (< 10 times). Check the result to make sure rare types are eliminated.

In [90]:
type_counts = ins['type'].value_counts()
common_types = type_counts[type_counts > 10].index.tolist()
ins2 = ins[ins['type'].isin(common_types)]
ins2['type'].value_counts()

type
routine    15429
Name: count, dtype: int64

In [92]:
ins2.head(50)

,business_id,score,date,type
0,10,82,20160503,routine
1,10,94,20140729,routine
2,10,92,20140114,routine
3,19,94,20160513,routine
4,19,94,20141110,routine
5,19,94,20140214,routine
6,24,98,20161005,routine
7,24,96,20160311,routine
8,24,96,20141124,routine
9,24,96,20140612,routine


#### Question 5c

Since the data was stored in a .csv file, the dates are formatted as strings such as `20160503`. Once we read in the data, we would like to have dates in an appropriate format for analysis. Add a new column called `year` by capturing the first four characters of the date column. 

Hint: we have seen multiple ways of doing this in class, includings `str` operations, `lambda` functions, `datetime` operations, and others. Choose the method that works best for you :)

In [93]:
ins['date'] = ins['date'].astype(str)
ins['year'] = ins['date'].str[:4]
print(ins.head())

   business_id  score      date     type  year
0           10     82  20160503  routine  2016
1           10     94  20140729  routine  2014
2           10     92  20140114  routine  2014
3           19     94  20160513  routine  2016
4           19     94  20141110  routine  2014


#### Question 5d

What range of years is covered in this data set? Are there roughly same number of inspections each year? Try dropping records for any years with less than 50 inspections and store the result in a new dataframe named `ins3`.

In [98]:
inspection_counts = ins['year'].value_counts()
years_with_enough_data = inspection_counts[inspection_counts > 50].index.tolist()
ins3 = ins[ins['year'].isin(years_with_enough_data)]
print (inspection_counts)
print(ins3['year'].value_counts())

year
2016    5840
2014    5629
2015    3923
2013      38
Name: count, dtype: int64
year
2016    5840
2014    5629
2015    3923
Name: count, dtype: int64


Let's examine only the inspections for one year: 2016. This puts businesses on a more equal footing because [inspection guidelines](https://www.sfdph.org/dph/eh/Food/Inspections.asp) generally refer to how many inspections should occur in a given year.

In [102]:
twenty_sixteen = ins3[ins3['year'] == '2016']
 
print(twenty_sixteen)

       business_id  score      date     type  year
0               10     82  20160503  routine  2016
3               19     94  20160513  routine  2016
6               24     98  20161005  routine  2016
7               24     96  20160311  routine  2016
10              31     84  20160816  routine  2016
...            ...    ...       ...      ...   ...
15425        89515     98  20161103  routine  2016
15426        89701    100  20161206  routine  2016
15427        89790     96  20161129  routine  2016
15428        89803    100  20161215  routine  2016
15429        89900     93  20161206  routine  2016

[5840 rows x 5 columns]


### Question 6

#### Question 6a

Merge the business and 2016 inspections data, keeping all businesses regardless of whether they show up in the inspections file. Show the first several rows of the resulting dataframe.

In [105]:
bus_2016ins = pd.merge(bus, twenty_sixteen, how = "outer", on = ["business_id", "business_id"])
bus_2016ins.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number,score,date,type,year
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044,82.0,20160503,routine,2016
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262,94.0,20160513,routine,2016
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,98.0,20161005,routine,2016
3,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,96.0,20160311,routine,2016
4,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN,84.0,20160816,routine,2016


#### Question 6b
Print the 20 lowest rated businesses names, their addresses, and their ratings.

In [108]:
low20 = bus_2016ins.nsmallest(20, ['score'])
low20.loc[:, ["name", "address", "score"]]

,name,address,score
4892,GOLDEN RIVER RESTAURANT,5827 GEARY BLVD,52.0
4873,UNCLE CAFE,65 WAVERLY PL,55.0
6281,CRAZY PEPPER,2257 SAN JOSE AVE,55.0
7325,POKI TIME,2101 LOMBARD ST,55.0
3008,HING WANG BAKERY,339 JUDAH ST,56.0
5627,GOLDEN WOK,295 B ORIZABA AVE,56.0
549,MA'S DIM SUM & CAFE,1315 POWELL ST,57.0
836,RED A BAKERY,634 CLEMENT ST,57.0
4209,SAN TUNG RESTAURANT LLC,1031 IRVING ST,57.0
4903,"NEW GARDEN RESTAURANT, INC.",716 KEARNY ST,57.0


## Done!

Now commit this notebook to your `hw3_submissions` branch, push it to your GitHub repo, and open a PR!